# Cap 10 Agregação de dados e operações de grupo

In [1]:
import numpy as np
import pandas as pd


## 10.1 Como considerar as operações de grupo

In [2]:
df = pd.DataFrame(
    {
        "key1": ["a", "a", None, "b", "b", "a", None],
        "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
        "data1": np.random.standard_normal(7),
        "data2": np.random.standard_normal(7),
    }
)


In [3]:
df


,key1,key2,data1,data2
0,a,1,0.909833,0.591758
1,a,2,1.390793,-0.212882
2,None,1,0.135419,0.463589
3,b,2,1.713500,-0.198974
4,b,1,0.220821,0.951279
5,a,<NA>,0.623154,-0.806617
6,None,1,-0.864718,0.003795


In [4]:
grouped = df["data1"].groupby(df["key1"])


In [5]:
grouped


In [6]:
grouped.mean()


key1
a    0.974593
b    0.967161
Name: data1, dtype: float64

In [7]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()


In [8]:
means


key1  key2
a     1       0.909833
      2       1.390793
b     1       0.220821
      2       1.713500
Name: data1, dtype: float64

In [9]:
means.unstack()


key2,1,2
key1,,
a,0.909833,1.390793
b,0.220821,1.713500


In [10]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])


In [11]:
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]


In [12]:
df["data1"].groupby([states, years]).mean()


CA  2005    1.006974
    2006    0.135419
OH  2005    1.311667
    2006   -0.321948
Name: data1, dtype: float64

In [13]:
df.groupby("key1").mean()


,key2,data1,data2
key1,,,
a,1.5,0.974593,-0.142580
b,1.5,0.967161,0.376152


In [14]:
df.groupby("key2").mean(numeric_only=True)


,data1,data2
key2,,
1,0.100339,0.502605
2,1.552147,-0.205928


<div style="border-left: 4px solid #ff5555; background: #282a36; color: #f8f8f2; padding: 10px; margin: 10px 0; font-family: sans-serif;">
  <strong>⚠️ Aviso:</strong>
  O uso de <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">numeric_only=True</code>
  é para evitar o erro que acontece porque a coluna <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">key2</code> está no tipo <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">Int64</code> *nullable* do Pandas, que não é tratado como <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">int64</code> puro. Na hora de fazer <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">.groupby("key2").mean()</code>, o Pandas tenta calcular a média também sobre a própria <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">key2</code>, mas como ela é internamente representada como <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">object</code> (mistura de inteiros e <code style="background-color: #44475a; color: #f8f8f2; padding: 2px 4px; border-radius: 4px;">pd.NA</code>), a operação falha.
</div>

In [15]:
df.groupby(["key1", "key2"]).mean()


data1     data2
key1 key2                    
a    1     0.909833  0.591758
     2     1.390793 -0.212882
b    1     0.220821  0.951279
     2     1.713500 -0.198974

In [16]:
df.groupby(["key1", "key2"]).size()


key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [17]:
df.groupby("key1", dropna=False).size()


key1
a      3
b      2
NaN    2
dtype: int64

In [18]:
df.groupby(["key1", "key2"], dropna=False).size()


key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [19]:
df.groupby("key1").count()


,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterando por grupos

In [20]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)


a
  key1  key2     data1     data2
0    a     1  0.909833  0.591758
1    a     2  1.390793 -0.212882
5    a  <NA>  0.623154 -0.806617
b
  key1  key2     data1     data2
3    b     2  1.713500 -0.198974
4    b     1  0.220821  0.951279


In [21]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)


('a', np.int64(1))
  key1  key2     data1     data2
0    a     1  0.909833  0.591758
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2  1.390793 -0.212882
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  0.220821  0.951279
('b', np.int64(2))
  key1  key2   data1     data2
3    b     2  1.7135 -0.198974


In [22]:
pieces = {name: group for name, group in df.groupby("key1")}


In [23]:
pieces["b"]


,key1,key2,data1,data2
3,b,2,1.713500,-0.198974
4,b,1,0.220821,0.951279


In [24]:
grouped = df.groupby(
    {"key1": "key", "key2": "key", "data1": "data", "data2": "data"},
    axis="columns",
)


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\1346027021.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby(


In [25]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)


data
      data1     data2
0  0.909833  0.591758
1  1.390793 -0.212882
2  0.135419  0.463589
3  1.713500 -0.198974
4  0.220821  0.951279
5  0.623154 -0.806617
6 -0.864718  0.003795
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### Seleção de uma coluna ou de um subconjunto de colunas

In [26]:
df.groupby(["key1", "key2"])[["data2"]].mean()


data2
key1 key2          
a    1     0.591758
     2    -0.212882
b    1     0.951279
     2    -0.198974

In [27]:
s_grouped = df.groupby(["key1", "key2"])["data2"]


In [28]:
s_grouped


In [29]:
s_grouped.mean()


key1  key2
a     1       0.591758
      2      -0.212882
b     1       0.951279
      2      -0.198974
Name: data2, dtype: float64

### Agrupamento com dicionários e Series

In [30]:
people = pd.DataFrame(
    np.random.standard_normal((5, 5)),
    columns=["a", "b", "c", "d", "e"],
    index=["Joe", "Steve", "Wanda", "Jill", "Trey"],
)


In [31]:
people.iloc[2:3, [1, 2]] = np.nan  # Adiona alguns valores NA


In [32]:
people


,a,b,c,d,e
Joe,-0.175272,0.232176,0.466758,1.408082,-1.130402
Steve,-0.221670,-0.385230,1.032285,-2.430113,-0.322917
Wanda,0.641114,NaN,NaN,-1.282385,-0.146702
Jill,0.230777,0.380263,1.243776,-1.455127,0.689966
Trey,0.764718,0.527871,0.036133,0.699766,-0.310452


In [33]:
mapping = {
    "a": "red",
    "b": "red",
    "c": "blue",
    "d": "blue",
    "e": "red",
    "f": "orange",
}


In [34]:
by_column = people.groupby(mapping, axis="columns")


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\2395188868.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


In [35]:
by_column.sum()


,blue,red
Joe,1.874840,-1.073498
Steve,-1.397827,-0.929818
Wanda,-1.282385,0.494412
Jill,-0.211351,1.301006
Trey,0.735899,0.982137


In [36]:
map_series = pd.Series(mapping)


In [37]:
map_series


a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [38]:
people.groupby(map_series, axis="columns").count()


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\3597945653.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Agrupamento com funções

In [39]:
people.groupby(len).sum()


,a,b,c,d,e
3,-0.175272,0.232176,0.466758,1.408082,-1.130402
4,0.995495,0.908134,1.279909,-0.755361,0.379514
5,0.419444,-0.385230,1.032285,-3.712497,-0.469620


In [40]:
key_list = ["one", "one", "one", "two", "two"]


In [41]:
people.groupby([len, key_list]).min()


,,a,b,c,d,e
3,one,-0.175272,0.232176,0.466758,1.408082,-1.130402
4,two,0.230777,0.380263,0.036133,-1.455127,-0.310452
5,one,-0.221670,-0.385230,1.032285,-2.430113,-0.322917


### Agrupamento por níveis de índice

In [42]:
columns = pd.MultiIndex.from_arrays(
    [["US", "US", "US", "JP", "JP"], [1, 3, 5, 1, 3]], names=["cty", "tenor"]
)


In [43]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)


In [44]:
hier_df


cty          US                            JP          
tenor         1         3         5         1         3
0     -0.114587 -0.025378 -0.070544  1.497678  2.215690
1     -0.439304  0.118561 -0.589545  0.273706  1.030465
2     -0.957535 -0.246714  0.380397  0.402937  0.053664
3     -0.752195  0.122705  1.343686  1.223607  0.760690

In [45]:
hier_df.groupby(level="cty", axis="columns").count()


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\2253353235.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Agregação de dados

In [46]:
df


,key1,key2,data1,data2
0,a,1,0.909833,0.591758
1,a,2,1.390793,-0.212882
2,None,1,0.135419,0.463589
3,b,2,1.713500,-0.198974
4,b,1,0.220821,0.951279
5,a,<NA>,0.623154,-0.806617
6,None,1,-0.864718,0.003795


In [47]:
grouped = df.groupby("key1")


In [48]:
grouped["data1"].nsmallest(2)


key1   
a     5    0.623154
      0    0.909833
b     4    0.220821
      3    1.713500
Name: data1, dtype: float64

In [49]:
def peak_to_peak(arr):
    return arr.max() - arr.min()


In [50]:
grouped.agg(peak_to_peak)


,key2,data1,data2
key1,,,
a,1,0.767639,1.398375
b,1,1.492679,1.150254


In [51]:
grouped.describe()


key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.974593  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.967161  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     1.150313  1.390793   3.0 -0.142580  0.701833 -0.806617 -0.509749   
b     1.340330  1.713500   2.0  0.376152  0.813352 -0.198974  0.088589   

                                    
           50%       75%       max  
key1                                
a    -0.212882  0.189438  0.591758  
b     0.376152  0.663716  0.951279  

[2 rows x 24 columns]

### Aplicação de uma função às colunas e aplicação de várias funções

In [52]:
tips = pd.read_csv("examples/tips.csv")


In [53]:
tips.head()


,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [54]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]


In [55]:
tips.head()


,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [56]:
grouped = tips.groupby(["day", "smoker"])


In [57]:
grouped_pct = grouped["tip_pct"]


In [58]:
grouped_pct.agg("mean")


day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [59]:
grouped_pct.agg(["mean", "std", peak_to_peak])


mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [60]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\3111280939.py:1: FutureWarning: The provided callable <function std at 0x0000020410CE6D40> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [61]:
functions = ["count", "mean", "max"]


In [62]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)


In [63]:
result


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [64]:
result["tip_pct"]


count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [65]:
ftuples = [("Average", "mean"), ("Variance", np.var)]


In [66]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\780895.py:1: FutureWarning: The provided callable <function var at 0x0000020410CE6E80> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [67]:
grouped.agg({"tip": np.max, "size": "sum"})


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\727700194.py:1: FutureWarning: The provided callable <function max at 0x0000020410CE6200> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip": np.max, "size": "sum"})


tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"], "size": "sum"})


tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Retorno de dados agregados sem índices de linha

In [69]:
tips.groupby(["day", "smoker"], as_index=False).mean(numeric_only=True)


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Método apply: dividir-aplicar-combinar genérico

In [70]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]


In [71]:
top(tips, n=6)


,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [72]:
tips.groupby("smoker").apply(top)


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\242339874.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [74]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\2722508240.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [75]:
result = tips.groupby("smoker")["tip_pct"].describe()


In [ ]:
result


,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [77]:
result.unstack("smoker")


       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### Supressão das chaves de grupo

In [78]:
tips.groupby("smoker", group_keys=False).apply(top)


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\2350148657.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Análise de quantis e de buckets

In [79]:
frame = pd.DataFrame(
    {
        "data1": np.random.standard_normal(1000),
        "data2": np.random.standard_normal(1000),
    }
)


In [ ]:
frame.head()


,data1,data2
0,2.225669,-0.595536
1,1.684081,-0.040070
2,0.942223,-0.094059
3,0.156625,-0.454037
4,-0.446852,1.777791


In [81]:
quartiles = pd.cut(frame["data1"], 4)


In [ ]:
quartiles.head(10)


0      (1.744, 3.203]
1      (0.285, 1.744]
2      (0.285, 1.744]
3     (-1.173, 0.285]
4     (-1.173, 0.285]
5    (-2.638, -1.173]
6    (-2.638, -1.173]
7      (0.285, 1.744]
8     (-1.173, 0.285]
9     (-1.173, 0.285]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.638, -1.173] < (-1.173, 0.285] < (0.285, 1.744] < (1.744, 3.203]]

In [83]:
def get_stats(group):
    return pd.DataFrame(
        {
            "min": group.min(),
            "max": group.max(),
            "count": group.count(),
            "mean": group.mean(),
        }
    )


In [84]:
grouped = frame.groupby(quartiles)


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\4157715240.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


In [86]:
grouped.apply(get_stats)


min       max  count      mean
data1                                                      
(-2.638, -1.173] data1 -2.632305 -1.177066    119 -1.636426
                 data2 -2.676213  3.146226    119  0.030309
(-1.173, 0.285]  data1 -1.159125  0.284753    521 -0.337660
                 data2 -3.129227  3.370979    521 -0.039965
(0.285, 1.744]   data1  0.291640  1.740616    330  0.868729
                 data2 -2.850102  2.505399    330  0.020644
(1.744, 3.203]   data1  1.745181  3.203256     30  2.118872
                 data2 -1.107277  1.555786     30  0.149971

In [87]:
grouped.agg(["min", "max", "count", "mean"])


data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-2.638, -1.173] -2.632305 -1.177066   119 -1.636426 -2.676213  3.146226   
(-1.173, 0.285]  -1.159125  0.284753   521 -0.337660 -3.129227  3.370979   
(0.285, 1.744]    0.291640  1.740616   330  0.868729 -2.850102  2.505399   
(1.744, 3.203]    1.745181  3.203256    30  2.118872 -1.107277  1.555786   

                                  
                 count      mean  
data1                             
(-2.638, -1.173]   119  0.030309  
(-1.173, 0.285]    521 -0.039965  
(0.285, 1.744]     330  0.020644  
(1.744, 3.203]      30  0.149971

In [88]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)


In [ ]:
quartiles_samp.head()


0    3
1    3
2    3
3    2
4    1
Name: data1, dtype: int64

In [ ]:
grouped = frame.groupby(quartiles_samp)


In [ ]:
grouped.apply(get_stats)


min       max  count      mean
data1                                           
0     data1 -2.632305 -0.655141    250 -1.243718
      data2 -3.006948  3.241393    250 -0.046905
1     data1 -0.654524 -0.017861    250 -0.310070
      data2 -3.129227  2.198157    250 -0.024603
2     data1 -0.015332  0.591203    250  0.260093
      data2 -3.121199  3.370979    250 -0.005099
3     data1  0.591219  3.203256    250  1.212061
      data2 -1.841181  2.052632    250  0.052994

### Exemplo: preenchimento de valores ausentes com valores específicos de grupo

In [92]:
s = pd.Series(np.random.standard_normal(6))


In [93]:
s[::2] = np.nan


In [ ]:
s


0         NaN
1    0.707687
2         NaN
3   -0.688613
4         NaN
5    2.044139
dtype: float64

In [95]:
s.fillna(s.mean())


0    0.687738
1    0.707687
2    0.687738
3   -0.688613
4    0.687738
5    2.044139
dtype: float64

In [96]:
states = [
    "Ohio",
    "New York",
    "Vermont",
    "Florida",
    "Oregon",
    "Nevada",
    "California",
    "Idaho",
]


In [97]:
group_key = ["East", "East", "East", "East", "West", "West", "West", "West"]


In [98]:
data = pd.Series(np.random.standard_normal(8), index=states)


In [ ]:
data


Ohio         -1.067573
New York      0.342854
Vermont      -0.649595
Florida       1.330333
Oregon        0.812944
Nevada       -0.907679
California    0.294179
Idaho         0.106980
dtype: float64

In [100]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan


In [ ]:
data


Ohio         -1.067573
New York      0.342854
Vermont            NaN
Florida       1.330333
Oregon        0.812944
Nevada             NaN
California    0.294179
Idaho              NaN
dtype: float64

In [102]:
data.groupby(group_key).size()


East    4
West    4
dtype: int64

In [ ]:
data.groupby(group_key).count()


East    3
West    2
dtype: int64

In [ ]:
data.groupby(group_key).mean()


East    0.201871
West    0.553562
dtype: float64

In [105]:
def fill_mean(group):
    return group.fillna(group.mean())


In [106]:
data.groupby(group_key).apply(fill_mean)


East  Ohio         -1.067573
      New York      0.342854
      Vermont       0.201871
      Florida       1.330333
West  Oregon        0.812944
      Nevada        0.553562
      California    0.294179
      Idaho         0.553562
dtype: float64

In [107]:
fill_values = {"East": 0.5, "West": -1}


In [108]:
def fill_func(group):
    return group.fillna(fill_values[group.name])


In [109]:
data.groupby(group_key).apply(fill_func)


East  Ohio         -1.067573
      New York      0.342854
      Vermont       0.500000
      Florida       1.330333
West  Oregon        0.812944
      Nevada       -1.000000
      California    0.294179
      Idaho        -1.000000
dtype: float64

### Exemplo: amostragem aleatória e permutação

In [110]:
suits = ["H", "S", "C", "D"]  # Copas, Espadas, Paus, Ouros
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]

cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)


In [111]:
deck.head(13)


AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [112]:
def draw(deck, n=5):
    return deck.sample(n)


In [ ]:
draw(deck)


8D     8
4S     4
4H     4
KH    10
3D     3
dtype: int64

In [114]:
def get_suit(card):
    # a última letra é o naipe
    return card[-1]


In [115]:
deck.groupby(get_suit).apply(draw, n=2)


C  4C     4
   2C     2
D  3D     3
   AD     1
H  2H     2
   QH    10
S  3S     3
   2S     2
dtype: int64

In [116]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)


3C      3
5C      5
4D      4
3D      3
10H    10
8H      8
2S      2
10S    10
dtype: int64

### Exemplo: média ponderada de grupos e correlação

In [117]:
df = pd.DataFrame(
    {
        "category": ["a", "a", "a", "a", "b", "b", "b", "b"],
        "data": np.random.standard_normal(8),
        "weights": np.random.uniform(size=8),
    }
)


In [ ]:
df


,category,data,weights
0,a,-0.276803,0.028170
1,a,-1.017819,0.994897
2,a,-0.401158,0.998928
3,a,-1.791804,0.620896
4,b,0.428813,0.154369
5,b,-0.845233,0.592098
6,b,-0.592577,0.492077
7,b,1.024928,0.157131


In [119]:
grouped = df.groupby("category")


In [120]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])


In [121]:
grouped.apply(get_wavg)


C:\Users\derek\AppData\Local\Temp\ipykernel_9024\423584187.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


category
a   -0.958675
b   -0.404687
dtype: float64

In [122]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True, index_col=0)


In [ ]:
close_px.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [ ]:
close_px.tail(4)


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [125]:
def spx_corr(group):
    return group.corrwith(group["SPX"])


In [126]:
rets = close_px.pct_change().dropna()


In [127]:
def get_year(x):
    return x.year


In [128]:
by_year = rets.groupby(get_year)


In [129]:
by_year.apply(spx_corr)


,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [130]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])


In [131]:
by_year.apply(corr_aapl_msft)


2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Exemplo: regressão linear nos grupos

In [132]:
import statsmodels.api as sm


def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1
    result = sm.OLS(Y, X).fit()
    return result.params


In [133]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])


,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Transformação de grupos e GroupBys "não encapsulados"

In [134]:
df = pd.DataFrame({"key": ["a", "b", "c"] * 4, "value": np.arange(12.0)})


In [ ]:
df


,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [136]:
g = df.groupby("key")["value"]


In [ ]:
g.mean()


key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [138]:
def get_mean(group):
    return group.mean()


In [139]:
g.transform(get_mean)


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [ ]:
g.transform("mean")


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [141]:
def times_two(group):
    return group * 2


In [ ]:
g.transform(times_two)


0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [143]:
def get_rank(group):
    return group.rank(ascending=False)


In [ ]:
g.transform(get_rank)


0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [145]:
def normalize(x):
    return (x - x.mean()) / x.std()


In [ ]:
g.transform(normalize)


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [ ]:
g.apply(normalize)


key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [ ]:
g.transform("mean")


0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [150]:
normalized = (df["value"] - g.transform("mean")) / g.transform("std")


In [ ]:
normalized


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## 10.5 Tabelas dinâmicas e tabulação cruzada

In [ ]:
tips.head()


,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [156]:
tips.pivot_table(
    values=["size", "tip", "tip_pct", "total_bill"], index=["day", "smoker"]
)


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [157]:
tips.pivot_table(
    index=["time", "day"], columns="smoker", values=["tip_pct", "size"]
)


size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [158]:
tips.pivot_table(
    index=["time", "day"],
    columns="smoker",
    values=["tip_pct", "size"],
    margins=True,
)


size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [159]:
tips.pivot_table(
    index=["time", "smoker"],
    columns="day",
    values="tip_pct",
    aggfunc=len,
    margins=True,
)


day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [160]:
tips.pivot_table(
    index=["time", "size", "smoker"],
    columns="day",
    values="tip_pct",
    fill_value=0,
)


day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Tabulações cruzadas: crosstab

In [ ]:
from io import StringIO


In [162]:
data = """Sample Nationality Handedness
    1 USA Right-handed
    2 Japan Left-handed
    3 USA Right-handed
    4 Japan Right-handed
    5 Japan Left-handed
    6 Japan Right-handed
    7 USA Right-handed
    8 USA Left-handed
    9 Japan Right-handed
    10 USA Right-handed"""


In [163]:
data = pd.read_table(StringIO(data), sep="\s+")


In [ ]:
data


,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [165]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)


Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [166]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)


smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244